In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [0]:
cd /../kaggle/input

In [0]:
cd facial-keypoints-detection

In [0]:
df=pd.read_csv("IdLookupTable.csv")
sub=pd.read_csv("SampleSubmission.csv")

In [0]:
train.fillna(method='ffill', inplace=True)

In [0]:
train.isnull().sum()

In [0]:
df["FeatureName"].value_counts()

In [0]:
with zipfile.ZipFile('training.zip') as existing_zip:
    with existing_zip.open('training.csv') as traincsv:
        train=pd.read_csv(traincsv)

In [0]:
with zipfile.ZipFile('test.zip') as existing_zip:
    with existing_zip.open('test.csv') as testcsv:
        test=pd.read_csv(testcsv)

In [0]:
train[train.columns[0]].std()

In [0]:
train_x=np.empty((7049,96,96,1))
test_x=np.empty((1783,96,96,1))

In [0]:
for i in range(7049):
    trainn=train["Image"][i].split(" ")
    train1=[int(x) for x in trainn]
    train2=np.array(train1,dtype="float")
    train3=train2.reshape(96,96,1)
    
    train_x[i]=train3

In [0]:
for i in range(1783):
    testt=test["Image"][i].split(" ")
    test1=[int(x) for x in testt]
    test2=np.array(test1,dtype="float")
    test3=test2.reshape(96,96,1)
    
    test_x[i]=test3

In [0]:
train_x = train_x / 255
test_x = test_x /255

In [0]:
train_y=train.drop(['Image'],axis=1)

In [0]:
mean=[]
std=[]

In [0]:
for columns in train_y.columns:
    mean.append(train_y[columns].mean())
    std.append(train_y[columns].std())
    train_y[columns] = (train_y[columns] - train_y[columns].mean()) / train_y[columns].std()

In [0]:
for column in train_y.columns:
    train_y[column].fillna(value=train_y[column].mean(),inplace = True)

In [0]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6,(3,3), activation = 'relu', input_shape=(96,96,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(12,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(30,activation='relu')
    ])
model.summary()

In [0]:
model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['accuracy'])

In [0]:
model.fit(train_x,train_y,epochs = 10)

In [0]:
pred = model.predict(test_x)

In [0]:
pred.shape

In [0]:
lookid_list = list(df['FeatureName'])
imageID = list(df['ImageId']-1)
pre_list = list(pred)

In [0]:
rowid = df['RowId']

In [0]:
feature = []
for f in list(df['FeatureName']):
    feature.append(lookid_list.index(f))

In [0]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y] * std[y] + mean[y])

In [0]:
rowid = pd.Series(rowid,name = 'RowId')

In [0]:
loc = pd.Series(preded,name = 'Location')

In [0]:
submission = pd.concat([rowid,loc],axis = 1)

In [0]:
cd /../kaggle/working

In [0]:
submission.to_csv('submission.csv',index = False)